In [4]:
from flair.data import Sentence
from flair.nn import Classifier
import pandas as pd
import re
import nltk
import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matthewschultz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matthewschultz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
tweets = pd.read_csv('/Users/matthewschultz/Big_Data_Lab/dominion_voting/output/attempt.csv', low_memory=False)

What I need to clean after thr first attempt:
- need to do UTF-8 encoding and ASCII characters
- minimium text cleaning is the best, and it should be defended
    - Give reason 

In [14]:
def remove_punctuation(x):
  x = [word for word in x if len(word) > 2]
  new_list = [re.sub("/\\n|\,|\(|\)|\[|\]/g", "", word) for word in x]
  i_list = [re.split("\/", word) for word in new_list]
  flatten_list = lambda irregular_list:[element for item in irregular_list for element in flatten_list(item)] if type(irregular_list) is list else [irregular_list]
  list4 = flatten_list(i_list)
  final_list = " ".join([word for word in list4 if word != ""])
  final_list = final_list.translate(str.maketrans('', '', string.punctuation))
  return final_list


def clean_tweets(df):
    lis_index = []
    uncleaned_tweets = []
    list_of_tweets = []
    df = df.dropna(subset=['Title'])
    for index, row in df.iterrows():
        tweet = row['Title']
        
        tweet = re.sub(r'https?:\/\/.\S+', "", tweet)
        tweet = re.sub(r'#', '', tweet)
        tweet = re.sub(r'^RT[\s]+', '', tweet)
        # tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",tweet) if s])

        tweet = nltk.word_tokenize(tweet)
        tweet = [word.encode(encoding="ascii", errors='replace') for word in tweet]
        tweet = [word.decode() for word in tweet]
        tweet = [word for word in tweet if len(word) > 1]
        tweet = remove_punctuation(tweet)

        tweet = re.sub(" +", " ", tweet)
        tweet = tweet.rstrip().lstrip()
        
        lis_index.append(index)
        uncleaned_tweets.append(row['Title'])
        list_of_tweets.append(tweet)
        if index == 100:
          # list_of_tweets = [tweet for tweet in list_of_tweets if tweet != '']
          return pd.DataFrame({'index': lis_index, 'uncleaned': uncleaned_tweets, 'cleaned': list_of_tweets})
        

cleaned_tweets = clean_tweets(tweets)
cleaned_tweets.to_csv('/Users/matthewschultz/Big_Data_Lab/dominion_voting/output/cleaned_tweets.csv', index = None)

to fix the putting the label in the middle of the sentence, will want to use recursion and pass in the

Flair has a problem with tokenizing 
does not have a set n-gram that it looks through

Title case is a primer 


In [18]:
# i used recursion for this function because the sentence would update when each tag when be inserted in.
# as a result, i use recursion to update the position each time
def engineer_sentence(entity, tag, tweet, index=0):
    indices_object = re.finditer(pattern=r"\s{}\s".format(entity), string=tweet)
    indices = [[index.start() + 1, index.start() + len(entity) + 1] for index in indices_object]
    if index < len(indices):
        i = indices[index]
        tweet = tweet[:i[0]] + '[' + tweet[i[0]:i[1]] + ']/{}'.format(tag) + tweet[i[1]:]
        tweet = re.sub(r'\/{}\/'.format(tag), '/', tweet)
        tweet = re.sub(r'\[\[', '[', tweet)
        tweet = re.sub(r'\]\]', ']', tweet)
        return engineer_sentence(entity, tag, tweet, index + 1)
    else:
        return tweet

def create_ner_df(lis):
    tweet_id_ = []
    entity_ = []
    tag_ =  []
    engineered_sentences = []
    index = 0
    for tweet in lis:
        print(index)
        # tokenizes sentence to preform different n-grams 
        sentence = Sentence(tweet)

        # load the NER tagger
        tagger = Classifier.load('ner')

        # run NER over sentence
        tagger.predict(sentence)
        replace = " " + sentence.to_original_text() + " "
        for label in sentence.get_labels():
            replace = engineer_sentence(label.data_point.text, label.value, replace)
            replace = re.sub(r'/(\w*)]', '', replace)
            tweet_id_.append(index)
            entity_.append(label.data_point.text)
            tag_.append(label.value)

        engineered_sentences.append(replace)
        index = index + 1

    engineered_sentences_df = pd.DataFrame({"labeled_setence": engineered_sentences})
    count_df = pd.DataFrame({"tweet_id": tweet_id_,'entity': entity_, 'tag': tag_})
    return engineered_sentences_df, count_df

engineered_sentences_df, count_df = create_ner_df(cleaned_tweets['cleaned'].to_list())

0
2023-04-27 22:56:16,601 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
1
2023-04-27 22:56:20,991 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2
2023-04-27 22:56:24,879 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
3
2023-04-27 22:56:28,659 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
4
2023-04-27 22:56:32,599 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG,

In [17]:
count_df.to_csv('/Users/matthewschultz/Big_Data_Lab/dominion_voting/output/ner_output/ner_output.csv', index = None)
engineered_sentences_df.to_csv('/Users/matthewschultz/Big_Data_Lab/dominion_voting/output/ner_output/labelled_sentence_output.csv', index = None)

#freq_table_frequency = count_df.groupby(['entity', 'tag']).size().reset_index(name="frequency")

#print(freq_table_frequency.loc[freq_table_frequency['tag'].str.contains("LOC")].reset_index())